In [1]:
# Basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# NLP
import nltk
from soynlp.normalizer import *
from hanspell import spell_checker
from konlpy.tag import Okt

# ML
from lightgbm import LGBMClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_curve
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split, ShuffleSplit, GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score , recall_score, f1_score, roc_auc_score, roc_curve


#wordcloud
from wordcloud import WordCloud, STOPWORDS
import numpy as np
from PIL import Image

# 한글화
import matplotlib.pyplot as plt 
from matplotlib import rc
plt.rcParams['axes.unicode_minus'] = False #마이너스 부호 때문에 한글이 깨질 수가 있어 주는 설정
rc('font', family='Arial Unicode MS')
%matplotlib inline                   


In [4]:
df_review = pd.read_csv('review_all_noun_tknized_rmved_final.csv', encoding='utf-8')
df_review = df_review[['reviews_for_vectorize', 'label']]
df_review

,reviews_for_vectorize,label
0,상세 설명 들어가지 않아도 검색 결과 중량 가격 확인 되면 더 편리하겠습니다,1.0
1,필터 기능 잘 되어있어 편하게 쇼핑 함,1.0
2,배송 빨라요 할인 많아요,1.0
3,50 퍼 쿠폰 만원 빵빵 하게 주는 곳 처음 봄 주문 하면 6시간 만에 옴 앱 뭐,1.0
4,빠른 배송,1.0
...,...,...
11316,모바일 쉽게 살 있어서,1.0
11317,키우며 좋은 식 자재 대한 갈증 있는데 이렇게 건강하고 신선한 먹거리 쉽게 접 있...,1.0
11318,아는 동생 소개 알 게 된 마켓 컬리 늦게 게 아쉬울 정도 제품 퀄리티 완젼 좋고...,1.0
11319,샛별 배송 정말 훌륭합니다 컬리 라면 무조건 믿고 주문,1.0


In [5]:
# 남아있는 숫자 제거
df_review['reviews_for_vectorize'] = df_review['reviews_for_vectorize'].apply(lambda x: normalize(x))
df_review['reviews_for_vectorize'][3]

'퍼 쿠폰 만원 빵빵 하게 주는 곳 처음 봄 주문 하면 시간 만에 옴 앱 뭐'

In [6]:
# 맞춤법 검사
errors = []

def spell_check(text):
    try:
        result = spell_checker.check(text)
        return result.checked
    except:
        errors.append(df_review[df_review['reviews_for_vectorize'] == text].index[0])
        return text  

df_review['reviews_for_vectorize'] = df_review['reviews_for_vectorize'].apply(spell_check)

In [ ]:
df_review.to_csv('reviews_spellchecked_b4_tokenizing.csv')

---

In [2]:
df_review = pd.read_csv('reviews_spellchecked_b4_tokenizing.csv')

#### Morphs

In [4]:
# 토큰화 (형태소)
df_review['morphs_reviews_for_vectorize'] = [ Okt().morphs(text) for text in df_review['reviews_for_vectorize'] ]

In [5]:
df_review

,Unnamed: 0,reviews_for_vectorize,label,morphs_reviews_for_vectorize
0,0,상세 설명 들어가지 않아도 검색 결과 중량 가격 확인되면 더 편리하겠습니다,1.0,"[상세, 설명, 들어가지, 않아도, 검색, 결과, 중량, 가격, 확인, 되면, 더,..."
1,1,필터 기능 잘 되어있어 편하게 쇼핑함,1.0,"[필터, 기능, 잘, 되어있어, 편하게, 쇼핑, 함]"
2,2,배송 빨라요 할인 많아요,1.0,"[배송, 빨라요, 할인, 많아요]"
3,3,퍼 쿠폰 만원 빵빵하게 주는 곳 처음 봄 주문하면 시간 만에 옴 앱 뭐,1.0,"[퍼, 쿠폰, 만원, 빵빵, 하게, 주는, 곳, 처음, 봄, 주문, 하면, 시간, ..."
4,4,빠른 배송,1.0,"[빠른, 배송]"
...,...,...,...,...
11316,11316,모바일 쉽게 살 있어서,1.0,"[모바일, 쉽게, 살, 있어서]"
11317,11317,키우며 좋은 식 자재 대한 갈증 있는데 이렇게 건강하고 신선한 먹거리 쉽게 접 있고...,1.0,"[키우며, 좋은, 식, 자재, 대한, 갈증, 있는데, 이렇게, 건강하고, 신선한, ..."
11318,11318,아는 동생 소개 알 게 된 마켓 컬리 늦게 게 아쉬울 정도 제품 퀄리티 완전 좋고 ...,1.0,"[아는, 동생, 소개, 알, 게, 된, 마켓, 컬리, 늦게, 게, 아쉬울, 정도, ..."
11319,11319,샛별 배송 정말 훌륭합니다 컬리 라면 무조건 믿고 주문,1.0,"[샛별, 배송, 정말, 훌륭합니다, 컬리, 라면, 무조건, 믿고, 주문]"


In [6]:
#vectorize reviews
contents_for_vectorize_morphs = []

for content in df_review['morphs_reviews_for_vectorize']: 
    sentence = ""
    for word in content:
        sentence = sentence + ' ' + word
        
    contents_for_vectorize_morphs.append(sentence)

In [7]:
#TfidfVectorizer
vectorizer_tfidf_morphs = TfidfVectorizer(min_df = 0.0, analyzer="char", sublinear_tf=True, ngram_range=(1,3), max_features=15800)

vectorizer_tfidf_morphs.fit_transform(contents_for_vectorize_morphs)

#말뭉치
corpus_morphs = []

for word in vectorizer_tfidf_morphs.get_feature_names_out():
    corpus_morphs.append(word)

In [101]:
# Light Gradient Boosting Machine

vectorizer_tfidf_morphs = TfidfVectorizer(min_df = 0.0, analyzer="char", sublinear_tf=True, ngram_range=(1,3), max_features=8000)

#feature, label 분리
X_morphs = vectorizer_tfidf_morphs.fit_transform(contents_for_vectorize_morphs)
y_morphs = df_review['label']

#train, test data 분리
X_morphs_train, X_morphs_test, y_morphs_train, y_morphs_test = train_test_split(X_morphs, 
                                                                        y_morphs, 
                                                                        test_size=0.3, 
                                                                        random_state=13, 
                                                                        stratify=y_morphs)

#kfold(StratifiedKFold가 좋을까?)
skfold = StratifiedKFold(n_splits=5)


#평가지표
accuracy = []
f1 = []
auc = []

#test_data 평가지표
accuracy_test = []
f1_test = []
auc_test = []


params={}
params['objective']='binary'
params['metric']='binary_logloss'
params['is_unbalance']=False
params['boosting']='gbdt'
params['device_type']='gpu'

params['learning_rate']=0.3         # 일반적으로 0.001 ~ 0.3 사이. 낮을 수록 더 많은 트리 필요
params['lambda_l2']=5               # 0 이상. 높을수록 복잡성 감소
params['num_leaves']=500        
params['max_depth']=30             # 30에서 좋은 성능 보인 후 더 이상 올려도 개선 없음.
params['feature_fraction']=0.5      # 1보다 작을 시 과정합 방지
params['bagging_fraction']=0.9      # 과적합 방지
params['bagging_freq']=5            # 과적합 방지
params['min_data_in_leaf']=10       # 데이터셋 중간(20~50), 큰(50~100), 작은(5~20)
params['verbosity']=-1              # LGBM 학습시 정보 출력. -1은 생략. 1은 일반정보, 2는 디버깅정보
params['subsample']=0.7             # 부스팅 단계서 80%만 학습. 과적합 방지
params['n_estimators']=100

num_rounds = 5000

lgbm_morphs = LGBMClassifier(**params)


for train_idx, validation_idx in skfold.split(X_morphs_train, y=y_morphs_train):
        
    X_train = X_morphs_train[train_idx]
    X_validation = X_morphs_train[validation_idx]
    y_train = y_morphs_train.iloc[train_idx]
    y_validation = y_morphs_train.iloc[validation_idx]
        
    #fit_predict
    lgbm_morphs.fit(X_train, y_train)   #, eval_set=[(X_train, y_train), (X_validation, y_validation)], eval_metric='binary_logloss'
    pred_validation = lgbm_morphs.predict(X_validation)
        
    #accuracy, f1, auc
    accuracy.append(accuracy_score(pred_validation, y_validation))
    f1.append(f1_score(pred_validation, y_validation))
    auc.append(roc_auc_score(pred_validation, y_validation))
    
pred_test = lgbm_morphs.predict(X_morphs_test) 
        
#test_data accuracy, f1, auc
accuracy_test.append(accuracy_score(pred_test, y_morphs_test))
f1_test.append(f1_score(pred_test, y_morphs_test))
auc_test.append(roc_auc_score(pred_test, y_morphs_test))



In [102]:
#평가지표

print('accuracy of lgbm : {}'.format(sum(accuracy) / len(accuracy)))
print('-'*100)
print('f1 of lgbm : {}'.format(sum(f1) / len(f1)))
print('-'*100)
print('auc of lgbm : {}'.format(sum(auc) / len(auc)))

print('='*100)

print('accuracy_test of lgbm : {}'.format(accuracy_test))
print('-'*100)
print('f1_test of lgbm : {}'.format(f1_test))
print('-'*100)
print('auc_test of lgbm : {}'.format(auc_test))

accuracy of lgbm : 0.9170895070579613
----------------------------------------------------------------------------------------------------
f1 of lgbm : 0.9492090154034859
----------------------------------------------------------------------------------------------------
auc of lgbm : 0.8834025750343606
accuracy_test of lgbm : [0.9163968207241684]
----------------------------------------------------------------------------------------------------
f1_test of lgbm : [0.9491039426523297]
----------------------------------------------------------------------------------------------------
auc_test of lgbm : [0.8896071073478944]


---

#### WordCloud 전처리

In [39]:
pred_test

array([0., 1., 1., ..., 0., 1., 1.])

In [40]:
y_morphs_test

6204     0.0
9662     1.0
5139     1.0
4978     1.0
3760     1.0
        ... 
7805     1.0
5894     1.0
10568    0.0
3110     1.0
8655     1.0
Name: label, Length: 3397, dtype: float64

In [41]:
#y_morphs_test label과 예측 결과를 dataframe으로 변환
df_wordcloud = y_morphs_test.to_frame()
df_wordcloud['pred'] = pred_test
df_wordcloud['TF'] = df_wordcloud['label'] == df_wordcloud['pred']
df_wordcloud

,label,pred,TF
6204,0.0,0.0,True
9662,1.0,1.0,True
5139,1.0,1.0,True
4978,1.0,1.0,True
3760,1.0,1.0,True
...,...,...,...
7805,1.0,1.0,True
5894,1.0,1.0,True
10568,0.0,0.0,True
3110,1.0,1.0,True


In [43]:
#y_morphs_test로 분류된 '리뷰', 'label', 'pred_test(test_data예측결과)'를 dataframe으로 만든다.
review = []

for i in y_morphs_test.index:
    review.append(" ".join(df_review['morphs_reviews_for_vectorize'][i]))
    
df_wordcloud['review'] = review
df_wordcloud = df_wordcloud[['review', 'label', 'pred', 'TF']]
df_wordcloud

,review,label,pred,TF
6204,최악 이네,0.0,0.0,True
9662,매우 만족,1.0,1.0,True
5139,최고,1.0,1.0,True
4978,좋은 상품 많고 배송 빠르고,1.0,1.0,True
3760,장보기 간편하고 제품 만족합니다,1.0,1.0,True
...,...,...,...,...
7805,신선한 제품 안전하게 받아서 좋고 반찬 너무 맛있어요,1.0,1.0,True
5894,늘 잘 애용 있습니다,1.0,1.0,True
10568,로딩 너무 느려요 아직 첫 주문 밖에 하긴 했는데 칫솔 사려 고 가입 로딩 크리 고...,0.0,0.0,True
3110,제품 믿음 가고 젛 아,1.0,1.0,True
